<a href="https://colab.research.google.com/github/5amessi/quora-question-pairs-kaggle-competition-/blob/master/Quora_Question_Pairs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Import kaggle
!pip install kaggle
!mkdir .kaggle
import json
token = {"username":"mohamedkhamis","key":"db0c0b339a224fbaf0e7dc74ced697bd"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}

mkdir: cannot create directory ‘.kaggle’: File exists
- path is now set to: {/content}


In [0]:
#download data
!kaggle competitions download -c quora-question-pairs

  0% 0.00/4.95M [00:00<?, ?B/s]
100% 4.95M/4.95M [00:00<00:00, 80.7MB/s]
 52% 11.0M/21.2M [00:00<00:00, 36.1MB/s]
100% 21.2M/21.2M [00:00<00:00, 60.9MB/s]
100% 112M/112M [00:01<00:00, 80.2MB/s] 
100% 112M/112M [00:01<00:00, 96.9MB/s]


In [0]:
#un zip data
import zipfile
path = "/content/{/content}/competitions/quora-question-pairs/train.csv.zip"
zip_ref = zipfile.ZipFile(path, 'r')
zip_ref.extractall("")
zip_ref.close()
path = "/content/{/content}/competitions/quora-question-pairs/test.csv.zip"
zip_ref = zipfile.ZipFile(path, 'r')
zip_ref.extractall("")
zip_ref.close()
path = "/content/{/content}/competitions/quora-question-pairs/sample_submission.csv.zip"
zip_ref = zipfile.ZipFile(path, 'r')
zip_ref.extractall("")
zip_ref.close()

In [0]:
#Download Word2vec and Glove
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
!wget -c "http://nlp.stanford.edu/data/glove.6B.zip"


--2019-11-08 11:54:02--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.112.171
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.112.171|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  78.7MB/s    in 18s     

2019-11-08 11:54:20 (85.5 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [0]:
import zipfile
path = "/content/glove.6B.zip"
zip_ref = zipfile.ZipFile(path, 'r')
zip_ref.extractall("")
zip_ref.close()

In [0]:
#import libs
import keras
import pandas as pd
import nltk
import string
from keras.layers import *
from keras.preprocessing.sequence import *
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt
import re
from string import punctuation
from tqdm import tqdm
from keras.models import load_model
import tensorflow as tf
import numpy as np
import keras.backend as K
TBT = nltk.tokenize.TreebankWordTokenizer()
LEM = nltk.stem.WordNetLemmatizer()
nltk.download('stopwords')
nltk.download('wordnet')
p_stemmer = nltk.stem.PorterStemmer()


Using TensorFlow backend.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
EMBEDDING_DIM = 300
MAX_SEQUENCE_LENGTH = 15
dict = {}

In [0]:
def read_dataset(test = False):
    
    if test == False:
      train = pd.read_csv("/content/train.csv")
      train_y = train["is_duplicate"]

    else: 
      train =  pd.read_csv("/content/test.csv")
      train =  train.fillna('empty')
      train_y = train['test_id']
    

    train_x_q1 = train["question1"]
    train_x_q2 = train["question2"]
    
    train_x_q1, train_x_q2, train_y = preprocess(train_x_q1,train_x_q2,train_y)
    
    return train_x_q1,train_x_q2,train_y

In [0]:
def preprocess(q1,q2,y):
    q1_processed = []
    q2_processed = []
    y_processed = []
    
    for i in range(len(q1)):
      try:
        
        x1 = Clean_text(q1[i],remove_stop_words=True, lemma = True)
        x2 = Clean_text(q2[i],remove_stop_words=True, lemma = True)
       
        q1_processed.append(x1)
        q2_processed.append(x2)
        
        y_processed.append(y[i])
      
      except: 
        print(i,q1[i])
        print(i,q2[i])
    return q1_processed, q2_processed, y_processed 

In [0]:
def Clean_text(text, remove_stop_words=False, lemma = False):
  
    text = re.sub(r"(@[a-z]*)", "", text)#remove and word start with @
    text = re.sub(r"(&[a-z;]*)", "", text)#remove and word start with &
    text = re.sub(r"(http|https|ftp|ftps)\:\/\/[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(\/\S*)?", "", text)#remove LINKs
    text = re.sub(r"(www.[a-z.\/0-9]*)", " ", text)#remove LINKs
    #repare text
    text = re.sub(r"’", "'", text)
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"what's", "", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"I'm", "I am", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"e-mail", "email", text)
    text = re.sub(r"bestfriend", "best friend", text)
    text = re.sub(r"[^A-Za-z0-9]", " ", text)
    text = re.sub(r"[0-9]", " ", text)

    text = ''.join([c for c in text if c not in punctuation])#remove pun

    if remove_stop_words:#remove stop words
        stop_words = set(stopwords.words('english')) 
        text = text.split()
        text = [w for w in text if not w in stop_words]
        text = " ".join(text)
      
    
    if lemma:
        text = text.split()
        wn = nltk.WordNetLemmatizer()
        lemm_words = [wn.lemmatize(word) for word in text]
        text = " ".join(lemm_words)
        
    return(text)

In [0]:
data_Q1 , data_Q2 ,label = read_dataset()

105780 How can I develop android app?
105780 nan
201841 How can I create an Android app?
201841 nan
363362 nan
363362 My Chinese name is Haichao Yu. What English name is most suitable for me considering the pronounciation of my Chinese name?


In [0]:
#tokenization and indexing
tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(data_Q1+data_Q2)
question1_word_sequences = tokenizer.texts_to_sequences(data_Q1)
question2_word_sequences = tokenizer.texts_to_sequences(data_Q2)
word_index = tokenizer.word_index
vocab_size = len(tokenizer.word_index)+1
vocab_size

74885

In [0]:
#read word2vev
path = '/content/GoogleNews-vectors-negative300.bin.gz'
embedding_values = KeyedVectors.load_word2vec_format(path, binary=True)
words = embedding_values.index2word
EMBEDDING_FILE_word2vec = {}
for word in words:
  EMBEDDING_FILE_word2vec[word] = embedding_values.get_vector(word)
embedding_values = []
words = []

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
#read Glove
f = open('/content/glove.6B.300d.txt')
EMBEDDING_FILE_glove = {}
for line in f.readlines():
    value = line.split(' ')
    word = value[0]
    coef = np.array(value[1:],dtype = 'float32')
    EMBEDDING_FILE_glove[word] = coef

In [0]:
def embedding(word_index,embedding_values):
    all_embs = np.stack(embedding_values.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    all_embs = []
    embedding_matrix = np.random.normal(emb_mean, emb_std, (vocab_size, 300))
    embeddings_index = {}
    OFV=[]
    for word,i in tqdm(word_index.items()):
        values = embedding_values.get(word)
        if values is not None:
            embedding_matrix[i] = values
        else:
          OFV.append(word)
    print(len(OFV))
    print(" | ".join(i for i in OFV))
    return embedding_matrix



In [0]:
word_embedding_matrix = np.concatenate([embedding(word_index,EMBEDDING_FILE_word2vec),embedding(word_index,EMBEDDING_FILE_glove)], axis=-1)
# word_embedding_matrix = embedding(word_index,EMBEDDING_FILE_word2vec)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)
100%|██████████| 74884/74884 [00:00<00:00, 496659.16it/s]


31448
quora | instagram | a | whatsapp | and | snapchat | jio | narendra | nra | favourite | upsc | sbi | spotify | to | cgl | mbbs | tcs | kvpy | cambodia | gmat | colour | mistry | iim | ece | centre | ibps | redmi | kejriwal | cgpa | flipkart | ielts | btech | of | bitcoin | accenture | paytm | iits | ncr | demonetisation | elon | arvind | travelling | brexit | goswami | arnab | grey | manaphy | demonetize | xiaomi | hadoop | upvotes | enfield | officejet | airprint | ugc | irctc | chhattisgarh | obc | iims | mahabharata | quorans | aiims | bba | airbnb | wwii | manipal | isro | toefl | mtech | imei | iiit | balaji | aadhaar | bhopal | icse | bengaluru | ipcc | vlsi | demonetizing | icici | mnc | pilani | zuckerberg | wechat | programme | voldemort | minecraft | coursera | upvoted | arduino | vivekananda | cpec | omegle | shippuden | deloitte | tyre | adolf | aadhar | kapil | oneplus | iisc | cheque | organisation | django | lyft | snowden | bitsat | angularjs | snapdeal | kohli | p

100%|██████████| 74884/74884 [00:00<00:00, 436508.59it/s]


20542
kvpy | ibps | redmi | flipkart | paytm | demonetisation | brexit | sqrt | manaphy | demonetize | upvotes | officejet | airprint | quorans | tatkal | aadhaar | demonetizing | wechat | upvoted | cpec | omegle | ntse | aadhar | oneplus | bitsat | angularjs | snapdeal | fiitjee | aipmt | imessage | amcat | upvote | unfollow | viber | articleship | emoji | cooky | blowjob | urjit | ncipe | zenfone | magento | backlinks | designjet | udemy | wbjee | displaystyle | selfie | blockchain | intp | miui | elitmus | iitians | laravel | pornstars | iocl | nmims | skyrim | demonitization | vaping | dfrac | emojis | infj | infty | aliexpress | zomato | upwork | srcc | mushkil | shopify | iift | duolingo | gixxer | quoran | findout | selfies | tqwl | hairfall | demonetise | iitian | afcat | kickass | searh | myntra | baleno | intjs | udacity | etg | topcoder | vape | ckwl | fiverr | tensorflow | shobhit | montr | infp | downvote | iitjee | hotstar | uberx | comedk | venmo | demonetising | nicmar 

In [0]:
data_Q1 = pad_sequences(question1_word_sequences, maxlen=MAX_SEQUENCE_LENGTH,padding='pre',truncating='post')
data_Q2 = pad_sequences(question2_word_sequences, maxlen=MAX_SEQUENCE_LENGTH,padding='pre',truncating='post')

datax = np.stack((data_Q1, data_Q2), axis=1)

X_train, X_test, y_train, y_test = train_test_split(datax, label, test_size=0.2, random_state=50)
Q1_train = X_train[:,0]
Q2_train = X_train[:,1]
Q1_test = X_test[:,0]
Q2_test = X_test[:,1]

In [0]:
#Manhattan Distance
def exponent_neg_manhattan_distance(tensors):
    left, right = tensors[0] , tensors[1]
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

In [0]:
#Cosine Distance
def cosine(tensors):
        l,r = tensors[0] , tensors[1]
        num = K.sum((l * r), keepdims=True, axis=-1)
        den = K.sqrt(K.sum(K.square(l), keepdims=True, axis=-1)) * K.sqrt(K.sum(K.square(r), keepdims=True, axis=-1))
#         den = K.clip(den, min_value=1e-4, max_value=float('inf'))
        sim = num / den
        return sim

In [0]:
#Building the model
def Load_model():

    question1 = Input(shape=(MAX_SEQUENCE_LENGTH,))
    question2 = Input(shape=(MAX_SEQUENCE_LENGTH,))

    embedding_layer = Embedding(input_dim=len(word_index)+1,output_dim=600,
                            weights=[word_embedding_matrix], input_length=MAX_SEQUENCE_LENGTH)

    LSTM_ = Bidirectional(CuDNNLSTM(1024, kernel_regularizer=keras.regularizers.l2(0.2),return_sequences=False))
    
    q1 = embedding_layer(question1)
    q1 = LSTM_(q1)
    
    q2 = embedding_layer(question2)
    q2 = LSTM_(q2)
    
    q1q2 = concatenate([q1,q2])

    q1q2 = Dense(1,activation='tanh')(q1q2)
        
    malstm_distance = Lambda(exponent_neg_manhattan_distance) ([q1, q2])
    
    cosine_distance = Lambda(cosine) ([q1, q2])
    
    is_duplicate = Dense(1,activation='sigmoid')(concatenate([cosine_distance,malstm_distance,q1q2]))

    model = Model([question1, question2], is_duplicate)

    model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.adam(), metrics=['accuracy'])
    model.summary()
    return model
model = Load_model()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 15)           0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 15)           0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 15, 600)      44931000    input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) (None, 2048)         13320192    embedding_3[0][0]          

In [0]:
model.fit([Q1_train, Q2_train], np.asarray(y_train), epochs=10, verbose= 1,
                    validation_data=([Q1_test,Q2_test], np.asarray(y_test)),
                    batch_size=1024)

Train on 323429 samples, validate on 80858 samples
Epoch 1/10
323429/323429 [==============================] - 119s 367us/step - loss: 14.7171 - acc: 0.6955 - val_loss: 0.5664 - val_acc: 0.7390
Epoch 2/10
323429/323429 [==============================] - 116s 360us/step - loss: 0.5378 - acc: 0.7530 - val_loss: 0.5303 - val_acc: 0.7541
Epoch 3/10
323429/323429 [==============================] - 116s 360us/step - loss: 0.5087 - acc: 0.7709 - val_loss: 0.5283 - val_acc: 0.7610
Epoch 4/10
323429/323429 [==============================] - 116s 360us/step - loss: 0.4901 - acc: 0.7836 - val_loss: 0.5372 - val_acc: 0.7393
Epoch 5/10
323429/323429 [==============================] - 116s 360us/step - loss: 0.4752 - acc: 0.7932 - val_loss: 0.5225 - val_acc: 0.7698
Epoch 6/10
323429/323429 [==============================] - 116s 360us/step - loss: 0.4630 - acc: 0.8024 - val_loss: 0.5164 - val_acc: 0.7634
Epoch 7/10
323429/323429 [==============================] - 116s 360us/step - loss: 0.4550 - acc

In [0]:
model.fit([Q1_train, Q2_train], np.asarray(y_train), epochs=2, verbose= 1,
                    validation_data=([Q1_test,Q2_test], np.asarray(y_test)),
                    batch_size=1024)

Train on 323429 samples, validate on 80858 samples
Epoch 1/2
323429/323429 [==============================] - 116s 360us/step - loss: 0.3695 - acc: 0.8556 - val_loss: 0.5087 - val_acc: 0.7969
Epoch 2/2
323429/323429 [==============================] - 116s 360us/step - loss: 0.3663 - acc: 0.8568 - val_loss: 0.5125 - val_acc: 0.7950


In [0]:
model.fit([data_Q1, data_Q2], np.asarray(label), epochs=5, verbose= 1, batch_size=512)
model.save_weights("model0.h5")

In [0]:
test_Q1 , test_Q2 , test_id = read_dataset(test = True)

In [0]:
test_Q1 = tokenizer.texts_to_sequences(test_Q1)
test_Q2 = tokenizer.texts_to_sequences(test_Q2)

test_Q1 = pad_sequences(test_Q1, maxlen=MAX_SEQUENCE_LENGTH,padding='pre',truncating='post')
test_Q2 = pad_sequences(test_Q2, maxlen=MAX_SEQUENCE_LENGTH,padding='pre',truncating='post')

In [0]:
# model = load_model('/content/model0.h5')
predict1 = model.predict([test_Q1 , test_Q2])

In [0]:
sample_submission = pd.read_csv('/content/sample_submission.csv')
sample_submission["is_duplicate"] = predict1
sample_submission.to_csv('/content/gdrive/My Drive/sample_submission2.csv', index=False)